In [3]:
df = pd.read_csv('C:/Users/Srijan-DS/Documents/Projects/identify-profitable-customers/data/raw/raw.csv')
df.head()

,id,purchase_amount,asset_amount,average_ratio,personal_id_1,personal_id_2,age,area,job_type,phone,...,average_favorite_score,card_history_period,score_1,score_2,score_3,score_4,total_amount_1,total_amount_2,total_amount_3,important_customer
0,0,62978.0,64016,87.170255,1256,86,44,NaN,Salaried,1,...,0.010922,0.048340,0,7,0,0.0,58620,10590,1392,0
1,1,66790.0,64497,83.609102,666,86,30,5318.0,Salaried,1,...,1.687598,1.701259,0,0,6,0.0,70809,9575,889,0
2,2,57788.0,111404,58.986180,1621,120,36,5954.0,Self employed,1,...,0.001265,3.483625,0,9,6,0.0,55294,38319,779,0
3,3,59729.0,63983,72.702121,1731,51,27,7263.0,NaN,1,...,0.015441,3.338835,0,1,2,0.0,1972,9314,2081,0
4,4,50533.0,98623,82.218856,801,86,40,6270.0,NaN,1,...,0.001283,0.067355,0,2,0,0.0,4942,10484,1772,0


In [13]:
import pandas as pd
import numpy as np
import os,sys
import pickle

df = pd.read_csv('C:/Users/Srijan-DS/Documents/Projects/identify-profitable-customers/data/raw/raw.csv')

df.drop(['id'],axis=1,inplace=True)
df.drop(['important_customer'],axis=1,inplace=True)

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, RobustScaler

from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import VarianceThreshold

transformer_1 = ColumnTransformer(
    transformers = [('tnf1', SimpleImputer(strategy='mean'), [0]),
                    ('tnf2', SimpleImputer(strategy='median'), [6]),
                    ('tnf3', SimpleImputer(strategy='most_frequent'), [7,10,12,15,16,22])
                    ],
     remainder = 'passthrough'
).set_output(transform='pandas')

transformer_2 = ColumnTransformer(
    transformers = [('trf4', OneHotEncoder(sparse=False,drop='first',handle_unknown='ignore'),[2])],
    remainder = 'passthrough'
).set_output(transform='pandas')

transformer_3 = ColumnTransformer(
    transformers = [('trf5', RobustScaler(),[1,8,9,11,18,19,23,24,25]),
                   ('trf6', MinMaxScaler(),[0,2,3,4,5,6,7,10,12,13,14,15,16,17,20,21,22])
                   ],
    remainder = 'passthrough'
).set_output(transform='pandas')

transformer_4 = ColumnTransformer(
    transformers = [('trf6', VarianceThreshold(threshold=0.05),[i for i in range(df.shape[1])])],
    remainder = 'passthrough'
).set_output(transform='pandas')

pipe = Pipeline(
    [
        ('transformer_1', transformer_1),
        ('transformer_2', transformer_2)
        #('transformer_3', transformer_3)
        #('transformer_4', transformer_4)
    ]
)

df = pipe.fit_transform(df)

extracted_names = [col.split('__')[-1] for col in df.columns]

df.columns = extracted_names

pickle.dump(pipe, open('fe_pipe.pkl','wb'))
pickle.dump(df, open('data.pkl','wb'))

C:\Users\Srijan-DS\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [20]:
from sklearn.ensemble import RandomForestClassifier

In [21]:
df = pd.read_csv('C:/Users/Srijan-DS/Documents/Projects/identify-important-customers/data/raw/train_df.csv')

df.drop(['id'],axis=1,inplace=True)

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, RobustScaler

from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import VarianceThreshold

transformer_1 = ColumnTransformer(
    transformers = [('tnf1', SimpleImputer(strategy='mean'), [0]),
                    ('tnf2', SimpleImputer(strategy='median'), [6]),
                    ('tnf3', SimpleImputer(strategy='most_frequent'), [7,10,12,15,16,22])
                    ],
     remainder = 'passthrough'
).set_output(transform='pandas')

transformer_2 = ColumnTransformer(
    transformers = [('trf4', OneHotEncoder(sparse=False,drop='first'),[2])],
    remainder = 'passthrough'
).set_output(transform='pandas')

transformer_3 = ColumnTransformer(
    transformers = [('trf5', RobustScaler(),[1,8,9,11,18,19,23,24,25]),
                   ('trf6', MinMaxScaler(),[0,2,3,4,5,6,7,10,12,13,14,15,16,17,20,21,22])
                   ],
    remainder = 'passthrough'
).set_output(transform='pandas')

transformer_4 = ColumnTransformer(
    transformers = [('trf6', VarianceThreshold(threshold=0.05),[i for i in range(df.shape[1])])],
    remainder = 'passthrough'
).set_output(transform='pandas')

pipe = Pipeline(
    [
        ('transformer_1', transformer_1),
        ('transformer_2', transformer_2)
        #('transformer_3', transformer_3)
        #('transformer_4', transformer_4)
    ]
)

df = pipe.fit_transform(df)

extracted_names = [col.split('__')[-1] for col in df.columns]

df.columns = extracted_names

X = df.drop(['important_customer'],axis=1)
y = df['important_customer']

rf = RandomForestClassifier(class_weight={0:1,1:7}, max_depth = 25, min_samples_split = 20,
                             min_samples_leaf = 20)
rf.fit(X,y)


## save

pickle.dump(rf, open('model.pkl', 'wb'))

C:\Users\Srijan-DS\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [22]:
X.head()

,job_type_Self employed,purchase_amount,area,personal_card_2,personal_card_4,campaign_use,card_expired,score_4,asset_amount,average_ratio,personal_id_1,personal_id_2,age,phone,personal_card_1,personal_card_3,car,purchase_score,average_favorite_score,card_history_period,score_1,score_2,score_3,total_amount_1,total_amount_2,total_amount_3
0,0.0,62978.0,2994.0,0.0,1.0,0.0,0.0,0.0,64016,87.170255,1256,86,44,1,1,0,0,0,0.010922,0.048340,0,7,0,58620,10590,1392
1,0.0,66790.0,5318.0,0.0,0.0,0.0,0.0,0.0,64497,83.609102,666,86,30,1,0,1,0,705,1.687598,1.701259,0,0,6,70809,9575,889
2,1.0,57788.0,5954.0,0.0,0.0,2.0,1.0,0.0,111404,58.986180,1621,120,36,1,1,0,0,309,0.001265,3.483625,0,9,6,55294,38319,779
3,1.0,59729.0,7263.0,0.0,0.0,0.0,0.0,0.0,63983,72.702121,1731,51,27,1,0,0,0,740,0.015441,3.338835,0,1,2,1972,9314,2081
4,1.0,50533.0,6270.0,0.0,0.0,0.0,0.0,0.0,98623,82.218856,801,86,40,1,1,0,0,3,0.001283,0.067355,0,2,0,4942,10484,1772


In [24]:
pickle.dump(X, open('training_data_order.pkl','wb'))

In [26]:
pd.DataFrame([X.iloc[1,:]])

,job_type_Self employed,purchase_amount,area,personal_card_2,personal_card_4,campaign_use,card_expired,score_4,asset_amount,average_ratio,personal_id_1,personal_id_2,age,phone,personal_card_1,personal_card_3,car,purchase_score,average_favorite_score,card_history_period,score_1,score_2,score_3,total_amount_1,total_amount_2,total_amount_3
1,0.0,66790.0,5318.0,0.0,0.0,0.0,0.0,0.0,64497,83.609102,666,86,30,1,0,1,0,705,1.687598,1.701259,0,0,6,70809,9575,889


In [32]:
X.index.tolist()

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [36]:
X.iloc[1]

job_type_Self employed          0.0
purchase_amount             66790.0
area                         5318.0
personal_card_2                 0.0
personal_card_4                 0.0
campaign_use                    0.0
card_expired                    0.0
score_4                         0.0
asset_amount                  64497
average_ratio             83.609102
personal_id_1                   666
personal_id_2                    86
age                              30
phone                             1
personal_card_1                   0
personal_card_3                   1
car                               0
purchase_score                  705
average_favorite_score     1.687598
card_history_period        1.701259
score_1                           0
score_2                           0
score_3                           6
total_amount_1                70809
total_amount_2                 9575
total_amount_3                  889
Name: 1, dtype: object

In [38]:
rf.predict(pd.DataFrame([X.iloc[1]]))[0]

0